## Further Analysis
Further analyzing the results and visual verification of the re-ranking.

In [ ]:
# Imports
from matplotlib import pyplot as plt

### Visual Analysis
Function to plot the images given top_k images that should be plotted.

In [ ]:
def plot_image(index, df, city, configuration, top_k=5):
    DATASET_VAL_QUERY = os.path.join(configuration["data_path"], city, "query", "images")
    DATASET_VAL_MAP = os.path.join(configuration["data_path"], city, "database", "images")

    # Determine the begin and end idx
    begin_idx = 0
    end_idx = 0
    for city_key in configuration["data_lengths"]:
        end_idx += configuration["data_lengths"][city_key]
        if city_key == city:
            break
        begin_idx += configuration["data_lengths"][city_key]
    
    # Check to ensure it doesn't go out of bounds
    if begin_idx+index >= end_idx:
        raise ValueError(f"Out of bounds man, keep it between 0 and {end_idx-begin_idx-1} for {city}!")

    # Get the file names
    query_image_name = df.loc[begin_idx+index, 'query_id']
    gcl_file_names = []
    reranked_file_names = []
    if isinstance(top_k, list):
        gcl_file_names = list(df.loc[begin_idx+index, 'retrieved_ids'])
        gcl_file_names = [gcl_file_names[idx] for idx in top_k]
        reranked_file_names = list(df.loc[begin_idx+index, 'retrieved_ids'])
        reranked_file_names = [reranked_file_names[idx] for idx in top_k]
    else:
        gcl_file_names = list(df.loc[begin_idx+index, 'retrieved_ids'])[0:top_k]
        reranked_file_names = list(df.loc[begin_idx+index, 'retrieved_ids'])[0:top_k]

    # Load the images into memory using NumPy
    query_image = plt.imread(os.path.join(DATASET_VAL_QUERY, query_image_name+".jpg"))
    reranked_images = np.array([plt.imread(os.path.join(DATASET_VAL_MAP, name+".jpg")) for name in reranked_file_names])
    gcl_images = np.array([plt.imread(os.path.join(DATASET_VAL_MAP, name+".jpg")) for name in gcl_file_names])

    # If total length is larger than 5
    if len(gcl_file_names) > 5:
        # If there is more than 5, make top_k + 1 rows and 2 columns
        fig, axs = plt.subplots(len(gcl_file_names)+1, 2, figsize=(10, 30))
        plt.subplots_adjust(wspace=0.1, hspace=0.1) # adjust spacing between subplots

        # set titles
        axs[0, 0].set_title("Query")
        axs[1, 0].set_title("Re-Ranking")
        axs[1, 1].set_title("GCL Method")

        # set images
        axs[0, 0].imshow(query_image)
        axs[0, 0].axis('off')
        axs[0, 1].axis('off')
        for i in range(1, len(gcl_file_names)+1):
            if isinstance(top_k, list):
                axs[i, 0].set_ylabel(f"Index : {top_k[i-1]}")
                axs[i, 1].set_ylabel(f"Index : {top_k[i-1]}")
            else:
                axs[i, 0].set_ylabel(f"Index : {i-1}")
                axs[i, 1].set_ylabel(f"Index : {i-1}")
            axs[i, 0].imshow(reranked_images[i-1])
            axs[i, 1].imshow(gcl_images[i-1])
            axs[i, 0].set_yticks([])
            axs[i, 0].set_xticks([])
            axs[i, 1].set_yticks([])
            axs[i, 1].set_xticks([])
    else:
        # If there is less than or equal to 5, make 3 rows and top_k columns
        fig, axs = plt.subplots(3, len(gcl_file_names), figsize=(10, 5))
        plt.subplots_adjust(wspace=0.1, hspace=0.2) # adjust spacing between subplots

        # set titles
        axs[0, 0].set_title("Query")
        axs[1, 0].set_title("Re-Ranking")
        axs[2, 0].set_title("GCL Method")

        # set images
        axs[0, 0].imshow(query_image)
        for i in range(0, len(gcl_file_names)):
            if isinstance(top_k, list):
                axs[1, i].set_xlabel(f"Index : {top_k[i]}")
                axs[2, i].set_xlabel(f"Index : {top_k[i]}")
            else:
                axs[1, i].set_xlabel(f"Index : {i}")
                axs[2, i].set_xlabel(f"Index : {i}")
            axs[1, i].imshow(reranked_images[i])
            axs[2, i].imshow(gcl_images[i])
            axs[0, i].axis('off')
            axs[1, i].set_yticks([])
            axs[1, i].set_xticks([])
            axs[2, i].set_yticks([])
            axs[2, i].set_xticks([])

    plt.tight_layout()
    plt.show()